In [21]:
import requests
import json

# Base URL for the API
url = "https://www.marksandspencer.com/c/food-and-wine/cooking/recipes/search/"

# Define headers
headers = {
    "accept": "application/json, text/plain, */*",
    "content-type": "application/json;charset=UTF-8",
    "x-requested-with": "XMLHttpRequest",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36",
}

# Function to scrape links
def scrape_links(page_number, page_size=40):
    # Construct the JSON payload
    data = {
        "Page": page_number,
        "PageSize": page_size,
        "Query": "",
        "Filter": [
            {"Operation": "AND", "Sublist": []},
            {"Operation": "OR", "Sublist": []}
        ]
    }

    response = requests.post(url, headers=headers, json=data)

    print(f"Requesting page {page_number}, Status Code: {response.status_code}")
    if response.status_code == 200: #If links not working it will give statues code 405
        return response.json()  # Return the JSON response
    else:
        print(f"Failed to retrieve page {page_number}: {response.status_code}, Response: {response.text}")
        return None

# Function to extract links
def extract_links(data):
    if data and "Results" in data:
        links = []
        for item in data["Results"]:
            links.append(item.get("Url"))  # Adjust the key as per your findings
        return links
    return []

# Main scraping loop
all_links = []
total_pages = 5  # Adjust this based on how many pages you want to scrape

for page in range(1, total_pages + 1):
    data = scrape_links(page)
    links = extract_links(data)
    all_links.extend(links)

# Print the scraped links
if all_links:
    print("Scraped Links:")
    for link in all_links:
        print(link)
else:
    print("No links found.")


Requesting page 1, Status Code: 200
Requesting page 2, Status Code: 200
Requesting page 3, Status Code: 200
Requesting page 4, Status Code: 200
Requesting page 5, Status Code: 200
No links found.


In [ ]:
service = Service('C:/drivers/chromedriver.exe')  # Make sure this path is correct


In [34]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException

# Set up Chrome options
chrome_options = Options()
chrome_options.add_argument("--headless")  # Run headless if you don't need a UI
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# Initialize the WebDriver (update the path)
service = Service('C:/path/to/chromedriver.exe')  # Update this path
driver = webdriver.Chrome(service=service, options=chrome_options)

# Replace with the URL of the page you want to scrape
url = "https://www.marksandspencer.com/c/food-and-wine/cooking/search/?filters=[]#/"
driver.get(url)

# Give the page time to load
time.sleep(2)

# Scroll and collect links
collected_links = set()
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    try:
        # Find all links on the current page
        links = driver.find_elements(By.TAG_NAME, 'a')
        for link in links:
            href = link.get_attribute('href')
            if href:
                collected_links.add(href)

        # Scroll down to the bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # Wait for new content to load

        # Calculate new height and compare to last height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:  # If no new content is loaded, break the loop
            break
        last_height = new_height

    except NoSuchElementException:
        print("Element not found, breaking the loop.")
        break

# Output the collected links
for link in collected_links:
    print(link)

# Close the driver
driver.quit()


NoSuchDriverException: Message: Unable to locate or obtain driver for chrome; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors/driver_location
